In [54]:
import bql
import bqviz as bqv
import pandas as pd
from ipywidgets import *

bq = bql.Service()

def clean_df(univ, start, end):
    
    # Parameters:
    date_expression = 'range({}, {})'.format(start, end)
    params_dict = { 'dates': date_expression,
                    'currency': 'usd',
                    'fill': 'next'}
    
    # Data Items:
    px_open = bq.data.px_open(**params_dict)
    px_last = bq.data.px_last(**params_dict)
    px_low = bq.data.px_low(**params_dict)
    px_high = bq.data.px_high(**params_dict)

    items_dict = {'Open': px_open, 
                  'Last': px_last,
                   'Low': px_low, 
                  'High': px_high}

    # Universe:
    bq_univ = univ

    # Fetching data:
    request = bql.Request(bq_univ, items_dict)
    response = bq.execute(request)
    df = bql.combined_df(response)
    df.reset_index(inplace=True)
    df.drop(['CURRENCY', 'ID'], axis=1, inplace=True)
    df.set_index('DATE', inplace=True)
                         
    return (df)

# Parameter dictionary:
params_dict_1 = {'univ': 'IBM US Equity',
               'start': '2016-01-01',
               'end': '2017-01-01'}

# Fetching data:
df_ibm = clean_df(**params_dict_1)
df_ibm.head()

,Open,Last,Low,High
DATE,,,,
2016-01-01,135.60,135.95,134.240000,135.97
2016-01-02,135.60,135.95,134.240000,135.97
2016-01-03,135.60,135.95,134.240000,135.97
2016-01-04,135.60,135.95,134.240000,135.97
2016-01-05,136.76,135.85,134.850125,136.89


In [55]:
# Netflix DataFrame:
params_dict_2 = {'univ': 'NFLX US Equity',
               'start': '2016-01-01',
               'end': '2017-01-01'}

df_nflx = clean_df(**params_dict_2)
df_nflx.head()

,Open,Last,Low,High
DATE,,,,
2016-01-01,109.00,109.96,105.21,110.00
2016-01-02,109.00,109.96,105.21,110.00
2016-01-03,109.00,109.96,105.21,110.00
2016-01-04,109.00,109.96,105.21,110.00
2016-01-05,110.45,107.66,105.85,110.58


In [56]:
# Interactive Scatter Plot:
interscatter =  bqv.InteractiveScatterPlot(df_ibm, hide_controls=True)
interscatter.show()

In [57]:
# X-axis selection:
interscatter._xsel

interactive(children=(Dropdown(description='X Axis', options=('Open', 'Last', 'Low', 'High'), value='Open'), O…

In [58]:
# Y-axis selection:
interscatter._ysel

interactive(children=(Dropdown(description='Y Axis', options=('Open', 'Last', 'Low', 'High'), value='Open'), O…

In [59]:
# Data selection (DataFrame selection):
# Returns a DataFrame with your selected scatter plot values (drag):
interscatter.selected_data

""


In [69]:
# Joining multiple graphs:
interlineplot_ibm = bqv.InteractiveLinePlot(df_ibm, title='IBM US Equity')
interlineplot_nflx = bqv.InteractiveLinePlot(df_nflx, title='NFLX US Equity')

df_ibm_reset = df_ibm.reset_index()
interscatterplot_ibm = bqv.InteractiveScatterPlot(df_ibm_reset)

VBox([interlineplot_ibm.show(), interlineplot_nflx.show(), interscatterplot_ibm.show()])

In [71]:
# Summary Statistics + graphs:
def stats(df_1):
    dict = { 'Min': df_1.min(),
             'Max': df_1.max(),
             'Mean': df_1.mean(),
             'Stdev': df_1.std()}
    
    df_2 = pd.DataFrame(dict)
    df_2.dropna(axis=1,inplace=True)
    
    return df_2

bars = bqv.BarPlot()
interlineplot.links= [[bars, stats]]
VBox([interlineplot.show(), bars.show()])